# Lab 7

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github//afarbin/DATA1401-Spring-2020/blob/master/Labs/Lab-7/Lab-7.ipynb)

Here are the "Gradebook" classes from lecture. For this lab, you will use these classes and are encouraged to modify them as you need.

In [423]:
import numpy as np
import math

# Create some virtual classes
class base:
    __name=""
    
    def __init__(self,name):
        self.__name=name

    def name(self):
        return self.__name

class data(base):
    def __init__(self,name):
        base.__init__(self,name)
        
class alg(base):
    def __init__(self,name):
        base.__init__(self,name)

In [424]:
class grade(data):
    __value=0
    __numerical=True
    __gradebook_name=str()
    __letter_grades=["F-","F","F+","D-","D","D+","C-","C","C+","B-","B","B+","A-","A","A+"]
    
    def __init__(self,name,numerical=True,value=None,raw=True):
        if value:
            if isinstance(value,(int,float)):
                self.__numerical=True
            elif isinstance(value,str):
                self.__numerical=False
            self.set(value)
        else:            
            self.__numerical=numerical
            
        self.__gradebook_name=name
        data.__init__(self,name+" Grade Algorithm")        

        
    def set(self,value):
        if isinstance(value,(int,float)) and self.__numerical:
            self.__value=value
        elif isinstance(value,str) and not self.__numerical:
            if value in self.__letter_grades:
                self.__value=value
        else:
            print (self.name()+" Error: Bad Grade.")
            raise Exception
    
    def value(self):
        return self.__value
    
    def numerical(self):
        return self.__numerical
    
    def gradebook_name(self):
        return self.__gradebook_name
    
    def __str__(self):
        return self.__gradebook_name+": "+str(self.__value)

class student(data):
    __id_number=0
    __grades=dict()
    
    def __init__(self,first_name, last_name,id_number):
        self.__id_number=id_number
        self.__grades=dict()
        data.__init__(self,first_name+" "+last_name+" Student Data")

    def add_grade(self,a_grade,overwrite=False):
        if overwrite or not a_grade.gradebook_name() in self.__grades:
            self.__grades[a_grade.gradebook_name()]=a_grade
        else:
            print (self.name()+" Error Adding Grade "+a_grade.name()+". Grade already exists.")
            raise Exception
        

    def id_number(self):
        return self.__id_number
    
    def print_grades(self):
        for grade in self.__grades:
            print (self.__grades[grade])            
    
    def __str__(self):
        return str(self.__id_number)+"  "+str(self.__grades)
    
    def __getitem__(self,key):
        return self.__grades[key]
    


In [425]:
class calculator(alg):    
    def __init__(self,name):
        alg.__init__(self,name)

    def apply(self,a_grade_book):
        raise NotImplementedError


In [426]:
class grade_book(data):
    # New member class to hold arbitrary data associated with the class

    __data=dict()
    __students=dict()
    __assignments=list()
    
    def __init__(self,name):
        data.__init__(self,name+" Course Grade Book")
        self.__students=dict()
        self.__data=dict()
        
    # New method to access data
    def __getitem__(self,key):
        return self.__data[key]
            
    # New method to add data
    def __setitem__(self, key, value):
        self.__data[key] = value
        
    def add_student(self,a_student):
        self.__students[a_student.id_number()]=a_student

    # New method to allow iterating over students
    def get_students(self):
        return self.__students
    
    def assign_grade(self,key,a_grade):
        the_student=None
        try:
            the_student=self.__students[key]
        except:
            for id in self.__students:
                if key == self.__students[id].name():
                    the_student=self.__students[id]
                    break
        if the_student:
            the_student.add_grade(a_grade)
        else:
            print (self.name()+" Error: Did not find student.")
            
    def apply_calculator(self,a_calculator,**kwargs):
        # a_calculator object is instantiated when invoking the apply_calculator method
        a_calculator.apply(self,**kwargs)
        
    def summarize_all(self):
        #summarize all assignments for all students
        pass
    
    
    #Print functions
    def print_data(self):
        for k,v in self.__data.items():
            print (k,":",v)
 
    def print_grades(self,grade_name):
        if isinstance(grade_name,str):
            grade_names=list()
            grade_names.append(grade_name)
        else:
            grade_names=grade_name
                      
        for k,a_student in self.__students.items():
            print (a_student.name(),end=" ")
            for a_grade_name in grade_names:
                print (a_student[a_grade_name],end=" ")
            print()
            
    def print_students(self):    
        for k,a_student in self.__students.items():
            print (k, a_student.name())
            a_student.print_grades()
            print ("_______________________________________")
            
    def print_stats(self):
        #the __data dict can hold misc sata so make sure we print correct stats
        for k, v in self.__data.items():
            if isinstance (k,str):
                if("Mean" in k):
                    print(k+": ",str(round(v,2)))
                if("STD" in k):
                    print(k+": ",str(round(v,2)))
    
        
    

In [427]:
class uncurved_letter_grade_percent(calculator):
    __grades_definition=[ (.97,"A+"),
                          (.93,"A"),
                          (.9,"A-"),
                          (.87,"B+"),
                          (.83,"B"),
                          (.8,"B-"),
                          (.77,"C+"),
                          (.73,"C"),
                          (.7,"C-"),
                          (.67,"C+"),
                          (.63,"C"),
                          (.6,"C-"),
                          (.57,"F+"),
                          (.53,"F"),
                          (0.,"F-")]
    __max_grade=100.
    __grade_name=str()
    
    def __init__(self,grade_name,max_grade=100.):
        self.__max_grade=max_grade
        self.__grade_name=grade_name
        calculator.__init__(self,
                                  "Uncurved Percent Based Grade Calculator "+self.__grade_name+" Max="+str(self.__max_grade))
        
    def apply(self,a_grade_book,grade_name=None,**kwargs):
        if grade_name:
            pass
        else:
            grade_name=self.__grade_name
            
  
        #iterate through the students and grab the selected grades
        for k,a_student in a_grade_book.get_students().items():
            a_grade=a_student[grade_name]

            if not a_grade.numerical():
                print (self.name()+ " Error: Did not get a numerical grade as input.")
                raise Exception
    
            percent=a_grade.value()/self.__max_grade
        
            for i,v in enumerate(self.__grades_definition):
                if percent>=v[0]:
                    break
            
            
            #print('add Letter: '+ grade_name+" Letter" + self.__grades_definition[i][1] )
            
            new_name = grade_name.replace("sum","Uncurved Letter")
            a_student.add_grade(grade(new_name,value=self.__grades_definition[i][1]),overwrite=True)
            

In [428]:
class mean_std_calculator(calculator):
    def __init__(self):
        calculator.__init__(self,"Mean and Standard Deviation Calculator")
        
    def apply(self,a_grade_book,grade_name,**kwargs):
        grades=list()
        for k,a_student in a_grade_book.get_students().items():
            grades.append(a_student[grade_name].value())
        
        a_grade_book[grade_name+" Mean"] = np.mean(grades)
        a_grade_book[grade_name+" STD"] = math.sqrt(np.var(grades))
        
        #i didnt know where to put these calc and this is an easy place
        a_grade_book[grade_name+" Max"] = max(grades)
        a_grade_book[grade_name+" Min"] = min(grades)

## CSV Reader

*Exercise 1*: The data for a class are stored in a "camma separated values" (CSV) file name `Data1401-Grades.csv` in the directory of this lab. You can see the contents using the `cat` shell command:

In [429]:
!ls

Copy_of_Lab_7.ipynb Data1401-Grades.txt


In [430]:
!cat Data1401-Grades.txt

You will note that the first line has the names of the "columns" of data, and that subsequent lines (or "rows") have the data for each student, separated by cammas.

Recalling that in lecture we created a file reader, create a CSV reader function that takes a filename as input and returns data structure(s) that store the data in the file. Note that you are not allowed to use a library. The point here is for *you* to write the CSV reader. Some options for your data structures (pick one):

* A list of dictionaries, where each element of the list is corresponds to a row of data and the dictionaries are keyed by the column name. For example `data[5]["l3_5"]` corresponds to the 6th student's grade on lab 3 question 5.

* A list of lists (i.e. a 2-D array or matrix) and a dictionary, where each element of the "matrix" corresponds to a a specific grade for a specific student and the dictionary maps the name of the column to the column index. For example `data[5][column_names["l1_5"]]` corresponds to the 6th student's grade on lab 3 question 5.

* A dictionary of lists, where each element of the dictionary corresponds to a column of data and the lists contain the data in that column. For example `data["l3_5"][5]` corresponds to the 6th student's grade on lab 3 question 5.

* (Extra Credit) A class that simultaneously supports all of the above methods.

In [431]:
def csv_reader(filename, structureType="ListOfLists", header=True):
    outDataFormats = ["DictOfLists","ListOfLists", "ListOfDicts"]

    if (structureType in outDataFormats):

        f=open(filename,"r")

        if (header):
            first_line = f.readline().rstrip()
            fields=first_line.split(",")

        if (structureType=="DictOfLists"):
            data=dict()
            for keyNames in fields: 
                data[keyNames] = list()
        else:
            data=list() 
      
        line = f.readline().rstrip()

        while line:
            items=line.split(",")

            if (structureType=="ListOfDicts"):
                row=dict()
            else:
                row=list()

            colCount=0

            for item in items:
                try:
                    d=float(item)
                except ValueError:
                    d=item
                
                # Store according to the structure in use
                if (structureType=="ListOfDicts"):
                    # append the dictionary to the list
                    row[fields[colCount]]=d
                elif (structureType=="DictOfLists"):
                    data[fields[colCount]].append(d) 
                else:
                    row.append(d)
                colCount += 1
            
            if (structureType=="ListOfDicts"):
                #Adding this list as sublist in myDict
                data.append(row) 
            elif (structureType=="ListOfLists"):
                data.append(row)
            
            line = f.readline().rstrip()

        f.close()    
            
        if (structureType=="ListOfLists"):
            return data, fields
        else:
            return data
    else:
        return None

In [432]:
#Lists of Lists
dataListOfLists, dataKeys = csv_reader("Data1401-Grades.txt")
print("\nAs a List of Lists: \nKeys: \n ", dataKeys,'\nData:\n', '\n'.join(map(str, dataListOfLists)))

#Dict of Lists
dataAsDictOfLists = csv_reader("Data1401-Grades.txt","DictOfLists")
print("\nAs a Dict of Lists:")
for key, value in dataAsDictOfLists.items():
    print(key, ' : ', value)
    
    
#Lists of Dicts
dataListOfDicts = csv_reader("Data1401-Grades.txt","ListOfDicts")
print("\nAs a List of Dicts:")
for idx in range(len(dataListOfDicts)):
    print(dataListOfDicts[idx])
    
#Unsupported structure format
badData = csv_reader("Data1401-Grades.txt","Unsupported")
print("\nAs unsupported: \n", badData)


#Default parameters
dataAsListofLists, dataKeys = csv_reader("Data1401-Grades.txt")
print("\nDefaults:\n", dataKeys, '\n'.join(map(str, dataAsListofLists)))


As a List of Lists: 
Keys: 
  ['l1_n', 'l1_1', '12_n', 'l2_1', 'l2_2', 'l2_3', 'l2_4', 'l2_5', 'l2_6', 'l2_7', 'l3_n', 'l3_1', 'l3_2', 'l3_3', 'l3_4', 'l3_5', 'l3_6', 'l3_7', 'l3_8', 'l3_9', 'l3_10', 'l3_11', 'l3_12', 'l3_13', 'l3_14', 'l4_n', 'l4_1', 'l4_2', 'l4_3', 'l4_4', 'l4_5', 'l4_6', 'l4_7', 'l4_8', 'l4_9', 'l4_10', 'l4_11', 'q1_n', 'q1_1', 'e1_n', 'e1_1', 'e1_2', 'e1_3', 'e1_4', 'e1_5', 'e1_6', 'e1_7', 'e1_8', 'e1_9', 'e1_10', 'e1_11', 'e1_12', 'e1_13', 'e1_14', 'e1_15'] 
Data:
 [1.0, 10.0, 7.0, 0.0, 10.0, 10.0, 8.0, 10.0, 10.0, 10.0, 14.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 9.5, 15.0, 9.0, 9.0, 0.0, 9.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[1.0, 10.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 15.0, 0.0, 0.0, 0.0, 0.

## Creating a Gradebook

*Exercise 2:* In lecture we used pandas to read the CSV file and create the grade book. The example below works for the CSV file for this lab. Modify the code below to use your CSV reader instead.

In [433]:
#Read in data file and create a gradebook
class_data = csv_reader("Data1401-Grades.txt","ListOfDicts")

a_grade_book=grade_book("Data 1401")

for student_i in range(len(class_data)):
    #instantiate a student obj w/ an id
    a_student_0=student("Student",str(student_i),student_i)

    #use key value pairs of class_data to add grades to the student then add student to gradebook
    for key, grade_value in class_data[student_i].items():
        a_student_0.add_grade(grade(key,value=grade_value))

    a_grade_book.add_student(a_student_0)

a_grade_book.print_data()
a_grade_book.print_students()

0 Student 0 Student Data
l1_n: 1.0
l1_1: 10.0
12_n: 7.0
l2_1: 0
l2_2: 10.0
l2_3: 10.0
l2_4: 8.0
l2_5: 10.0
l2_6: 10.0
l2_7: 10.0
l3_n: 14.0
l3_1: 9.0
l3_2: 0
l3_3: 0
l3_4: 0
l3_5: 0
l3_6: 0
l3_7: 0
l3_8: 0
l3_9: 0
l3_10: 0
l3_11: 0
l3_12: 0
l3_13: 0
l3_14: 0
l4_n: 11.0
l4_1: 0
l4_2: 0
l4_3: 0
l4_4: 0
l4_5: 0
l4_6: 0
l4_7: 0
l4_8: 0
l4_9: 0
l4_10: 0
l4_11: 0
q1_n: 1.0
q1_1: 9.5
e1_n: 15.0
e1_1: 9.0
e1_2: 9.0
e1_3: 0
e1_4: 9.0
e1_5: 8.0
e1_6: 0
e1_7: 0
e1_8: 0
e1_9: 0
e1_10: 0
e1_11: 0
e1_12: 0
e1_13: 0
e1_14: 0
e1_15: 0
_______________________________________
1 Student 1 Student Data
l1_n: 1.0
l1_1: 10.0
12_n: 7.0
l2_1: 0
l2_2: 0
l2_3: 0
l2_4: 0
l2_5: 0
l2_6: 0
l2_7: 0
l3_n: 14.0
l3_1: 0
l3_2: 0
l3_3: 0
l3_4: 0
l3_5: 0
l3_6: 0
l3_7: 0
l3_8: 0
l3_9: 0
l3_10: 0
l3_11: 0
l3_12: 0
l3_13: 0
l3_14: 0
l4_n: 11.0
l4_1: 0
l4_2: 0
l4_3: 0
l4_4: 0
l4_5: 0
l4_6: 0
l4_7: 0
l4_8: 0
l4_9: 0
l4_10: 0
l4_11: 0
q1_n: 1.0
q1_1: 0
e1_n: 15.0
e1_1: 0
e1_2: 0
e1_3: 0
e1_4: 0
e1_5: 0
e1_6: 0
e1_7: 0
e1_8: 0
e

## Grade Summing

*Exercise 3:* In lectre we will change the design of our algorithm classes and then update the `uncurved_letter_grade_percent` calculator. In lecture we also created a `grade_summer` calcuator that takes a prefix (for example `e1_` and a number `n`) and sums all grades starting with that prefix up to `n` and creates a new sum grade. Update this calculator (below) to the new design of our algorithm classes. Test your updated calculator by using it to sum the grades for all labs, quizzes, and exams of each student.

In [434]:
#updated
class grade_summer(calculator):
    def __init__(self,prefix,n=None):
        self.__prefix=prefix
        self.__n=n
        calculator.__init__(self,"Sum Grades")
        
    def apply(self,a_grade_book,**kwargs):
        first=True
        
        #print('prefix = ', self.__prefix, ' n= ', str(self.__n))

        for k,a_student in a_grade_book.get_students().items():
            if first:
                first=False                
                if self.__n:
                    labels=[self.__prefix+str(x) for x in range(1,self.__n)]
                else:
                    labels=list()
                    for i in range(1,1000):
                        label=self.__prefix+str(i)
                        try:
                            a_grade=a_student[label]
                            labels.append(label)
                        except:
                            break                

            grade_sum=0.
            for label in labels:
                grade_sum+=a_student[label].value()

            a_student.add_grade(grade(self.__prefix+"sum",value=grade_sum),**kwargs)

In [435]:
#checking that grade summerization calc works (all exercises from one lab)
a_grade_book.apply_calculator(grade_summer("l2_"),overwrite=True)
a_grade_book.print_grades("l2_sum")

Student 0 Student Data l2_sum: 58.0 
Student 1 Student Data l2_sum: 0 
Student 2 Student Data l2_sum: 0 
Student 3 Student Data l2_sum: 62.0 
Student 4 Student Data l2_sum: 49.5 
Student 5 Student Data l2_sum: 69.0 
Student 6 Student Data l2_sum: 54.5 
Student 7 Student Data l2_sum: 69.0 
Student 8 Student Data l2_sum: 39.5 
Student 9 Student Data l2_sum: 60.0 
Student 10 Student Data l2_sum: 47.0 
Student 11 Student Data l2_sum: 68.5 
Student 12 Student Data l2_sum: 64.0 
Student 13 Student Data l2_sum: 49.5 
Student 14 Student Data l2_sum: 68.0 
Student 15 Student Data l2_sum: 59.0 


In [436]:
#summing grades for all labs, quizzes, exams of each student
a_grade_book.apply_calculator(grade_summer("l1_"),overwrite=True)
a_grade_book.apply_calculator(grade_summer("l2_"),overwrite=True)
a_grade_book.apply_calculator(grade_summer("l3_"),overwrite=True)
a_grade_book.apply_calculator(grade_summer("l4_"),overwrite=True)

a_grade_book.apply_calculator(grade_summer("e1_"),overwrite=True)

a_grade_book.apply_calculator(grade_summer("q1_"),overwrite=True)

In [437]:
# uncurved calculator against the summed scores
a_grade_book.apply_calculator(uncurved_letter_grade_percent("l1_sum",max_grade=10))
a_grade_book.apply_calculator(uncurved_letter_grade_percent("l2_sum",max_grade=70))
a_grade_book.apply_calculator(uncurved_letter_grade_percent("l3_sum",max_grade=140))
a_grade_book.apply_calculator(uncurved_letter_grade_percent("l4_sum",max_grade=110))
a_grade_book.apply_calculator(uncurved_letter_grade_percent("q1_sum",max_grade=10))
a_grade_book.apply_calculator(uncurved_letter_grade_percent("e1_sum",max_grade=150))

In [438]:
a_grade_book.print_grades("l1_sum")
print("------------------")
a_grade_book.print_grades("l2_sum")
print("------------------")
a_grade_book.print_grades("l3_sum")
print("------------------")
a_grade_book.print_grades("l4_sum")
print("------------------")
a_grade_book.print_grades("q1_sum")
print("------------------")
a_grade_book.print_grades("e1_sum")

Student 0 Student Data l1_sum: 10.0 
Student 1 Student Data l1_sum: 10.0 
Student 2 Student Data l1_sum: 10.0 
Student 3 Student Data l1_sum: 10.0 
Student 4 Student Data l1_sum: 10.0 
Student 5 Student Data l1_sum: 10.0 
Student 6 Student Data l1_sum: 10.0 
Student 7 Student Data l1_sum: 10.0 
Student 8 Student Data l1_sum: 10.0 
Student 9 Student Data l1_sum: 10.0 
Student 10 Student Data l1_sum: 10.0 
Student 11 Student Data l1_sum: 10.0 
Student 12 Student Data l1_sum: 10.0 
Student 13 Student Data l1_sum: 10.0 
Student 14 Student Data l1_sum: 10.0 
Student 15 Student Data l1_sum: 10.0 
------------------
Student 0 Student Data l2_sum: 58.0 
Student 1 Student Data l2_sum: 0 
Student 2 Student Data l2_sum: 0 
Student 3 Student Data l2_sum: 62.0 
Student 4 Student Data l2_sum: 49.5 
Student 5 Student Data l2_sum: 69.0 
Student 6 Student Data l2_sum: 54.5 
Student 7 Student Data l2_sum: 69.0 
Student 8 Student Data l2_sum: 39.5 
Student 9 Student Data l2_sum: 60.0 
Student 10 Student 

## Curving Grades

*Exercise 4:* Use the `mean_std_calculator` above to calculate the mean and standard deviation for every lab, quiz, and exam in the class. Add a new print function to the `grade_book` class to print out such information in a nice way, and use this function to show your results.


In [439]:
# Your solution here
a_grade_book.apply_calculator(mean_std_calculator(), grade_name="l1_sum")
a_grade_book.apply_calculator(mean_std_calculator(), grade_name="l2_sum")
a_grade_book.apply_calculator(mean_std_calculator(), grade_name="l3_sum")
a_grade_book.apply_calculator(mean_std_calculator(), grade_name="l4_sum")
a_grade_book.apply_calculator(mean_std_calculator(), grade_name="e1_sum")
a_grade_book.apply_calculator(mean_std_calculator(), grade_name="q1_sum")

a_grade_book.print_stats()

l1_sum Mean:  10.0
l1_sum STD:  0.0
l2_sum Mean:  51.09
l2_sum STD:  21.06
l3_sum Mean:  73.31
l3_sum STD:  38.3
l4_sum Mean:  34.12
l4_sum STD:  30.42
e1_sum Mean:  82.69
e1_sum STD:  42.94
q1_sum Mean:  6.41
q1_sum STD:  4.47


*Exercise 5:* In lecture we will change the design of our algorithms classes and then update the `uncurved_letter_grade_percent` calculator. Do the same for the `curved_letter_grade` calculator below and by curving all the lab, quiz, and exam grades.

In [440]:
class curved_letter_grade(calculator):
    __grades_definition=[ (.97,"A+"),
                          (.93,"A"),
                          (.9,"A-"),
                          (.87,"B+"),
                          (.83,"B"),
                          (.8,"B-"),
                          (.77,"C+"),
                          (.73,"C"),
                          (.7,"C-"),
                          (.67,"C+"),
                          (.63,"C"),
                          (.6,"C-"),
                          (.57,"F+"),
                          (.53,"F"),
                          (0.,"F-")]
    __max_grade=100.
    __grade_name=str()
    
    def __init__(self,grade_name,mean,std,max_grade=100.):
        self.__max_grade=max_grade
        self.__mean=mean
        self.__std=std
        self.__grade_name=grade_name
        calculator.__init__(self,
                                  "Curved Percent Based Grade Calculator "+self.__grade_name+ \
                                  " Mean="+str(self.__mean)+\
                                  " STD="+str(self.__std)+\
                                  " Max="+str(self.__max_grade))
        

    def apply(self,a_grade,grade_name=None, **kwargs):
        if grade_name:
            pass
        else:
            grade_name=self.__grade_name
            
        #print('maxgrade:', self.__max_grade,'mean: ',self.__mean, 'std: ',self.__std)
        #iterate through the students and grab the selected grades
        for k, a_student in a_grade_book.get_students().items():
            a_grade=a_student[grade_name]
            
            if not a_grade.numerical():
                print(self.name()+" Error: Did not get a numerical grade as input")
                raise Exception
                
            percent=a_grade.value()/self.__max_grade
            shift_to_zero=percent-(self.__mean/self.__max_grade)
            if (self.__std/self.__max_grade)!=0:
                scale_std=0.1*shift_to_zero/(self.__std/self.__max_grade)
                scaled_percent=scale_std+0.8
            else:
                #handle case if std was 0 leading to Nan problems
                scaled_percent=percent
        
        #print(a_student.id_number(),'gradename=',grade_name,'value=',a_grade.value(),
                #'%:',round(percent,2), '<0:',round(shift_to_zero,2),'scaleSTD:',scale_std,'scale%:',scaled_percent)
        
        for i,v in enumerate(self.__grades_definition):
            if scaled_percent>=v[0]:
                break
    
    
        new_name = grade_name.replace("sum","Curved Letter")                    
        a_student.add_grade(grade(new_name,value=self.__grades_definition[i][1]))
        

In [441]:
a_grade_book.apply_calculator(curved_letter_grade("l1_sum",a_grade_book["l1_sum Mean"],a_grade_book["l1_sum STD"],max_grade=10))
a_grade_book.apply_calculator(curved_letter_grade("l2_sum",a_grade_book["l2_sum Mean"],a_grade_book["l2_sum STD"],max_grade=140))
a_grade_book.apply_calculator(curved_letter_grade("l3_sum",a_grade_book["l3_sum Mean"],a_grade_book["l3_sum STD"],max_grade=110))
a_grade_book.apply_calculator(curved_letter_grade("l4_sum",a_grade_book["l4_sum Mean"],a_grade_book["l4_sum STD"],max_grade=10))
a_grade_book.apply_calculator(curved_letter_grade("q1_sum",a_grade_book["q1_sum Mean"],a_grade_book["q1_sum STD"],max_grade=150))
a_grade_book.apply_calculator(curved_letter_grade("e1_sum",a_grade_book["e1_sum Mean"],a_grade_book["e1_sum STD"],max_grade=100))

In [442]:
a_grade_book.print_grades("l1_Curved Letter")
print("---------------------")
a_grade_book.print_grades("l2_Curved Letter")
print("---------------------")
a_grade_book.print_grades("l3_Curved Letter")
print("---------------------")
a_grade_book.print_grades("l4_Curved Letter")
print("---------------------")
a_grade_book.print_grades("q1_Curved Letter")
print("---------------------")
a_grade_book.print_grades("e1_Curved Letter")

Student 0 Student Data 

KeyError: 'l1_Curved Letter'

## Final Course Grade

*Exercise 6:* Write a new calculator that sums grades with a prefix, as in the `grade_summer` calculator, but drops `n` lowest grades. Apply the algorithm to drop the lowest lab grade in the data.


In [443]:
class grade_final(calculator):
    def __init__(self,prefix,drop=0):
        self.__prefix=prefix
        self.__drop=drop
        calculator.__init__(self,"Final Grades")
        
    def apply(self,a_grade_book,**kwargs):
        # for each student in the grade book

        for k,a_student in a_grade_book.get_students().items():
            labels=list()
            for i in range(1,1000):
                label=self.__prefix+str(i)+"_sum"                      
                try:
                    a_grade=a_student[label]
                    labels.append(label)
                except:
                    break                

            grade_list=list()
            for label in labels:
                grade_list.append(a_student[label].value())

            # drop lowest grades defined by self.__drop but only if the number of grades is bigger than the drop number
            if (len(grade_list)>self.__drop):
                for x in range(self.__drop):
                    grade_list.remove(min(grade_list))
                
            grade_sum=0
            for iterG in grade_list:
                grade_sum+=iterG

            a_student.add_grade(grade(self.__prefix+"_final",value=grade_sum),**kwargs)

In [444]:
# Generate the final lab grades, w/ out dropping for comparison
a_grade_book.apply_calculator(grade_final("l"),overwrite=True)
a_grade_book.print_grades("l_final")

Student 0 Student Data l_final: 77.0 
Student 1 Student Data l_final: 10.0 
Student 2 Student Data l_final: 81.0 
Student 3 Student Data l_final: 249.0 
Student 4 Student Data l_final: 163.0 
Student 5 Student Data l_final: 203.0 
Student 6 Student Data l_final: 159.0 
Student 7 Student Data l_final: 170.0 
Student 8 Student Data l_final: 157.5 
Student 9 Student Data l_final: 123.0 
Student 10 Student Data l_final: 288.0 
Student 11 Student Data l_final: 227.0 
Student 12 Student Data l_final: 189.0 
Student 13 Student Data l_final: 199.0 
Student 14 Student Data l_final: 173.0 
Student 15 Student Data l_final: 228.0 


In [445]:
# Generate the final lab grades, dropping the lowest
a_grade_book.apply_calculator(grade_final("l",drop=1),overwrite=True)
a_grade_book.print_grades("l_final")

Student 0 Student Data l_final: 77.0 
Student 1 Student Data l_final: 10.0 
Student 2 Student Data l_final: 81.0 
Student 3 Student Data l_final: 239.0 
Student 4 Student Data l_final: 153.0 
Student 5 Student Data l_final: 193.0 
Student 6 Student Data l_final: 149.0 
Student 7 Student Data l_final: 160.0 
Student 8 Student Data l_final: 147.5 
Student 9 Student Data l_final: 113.0 
Student 10 Student Data l_final: 278.0 
Student 11 Student Data l_final: 217.0 
Student 12 Student Data l_final: 189.0 
Student 13 Student Data l_final: 189.0 
Student 14 Student Data l_final: 173.0 
Student 15 Student Data l_final: 218.0 


*Exercise 7*: Write a new calculator that creates a new letter grade based on a weighted average of letter grades, by assigning the following numerical values to letter grades:

In [446]:
GradeMap={"A+":12,
          "A":11,
          "A-":10,
          "B+":9,
          "B":8,
          "B-":7,
          "C+":6,
          "C":5,
          "C-":4,
          "D+":3,
          "D":2,
          "D-":1,
          "F":0}

In [447]:
class weighted_letter_grade(calculator):
    __grades_definition=[ {"A+":12,
                           "A":11,
                           "A-":10,
                           "B+":9,
                           "B":8,
                           "B-":7,
                           "C+":6,
                           "C":5,
                           "C-":4,
                           "D+":3,
                           "D":2,
                           "D-":1,
                           "F+":0,
                           "F":0,
                           "F-":0}
                         ]

    def __init__(self,grade_name):
        self.__max_grade=max_grade
        self.__grade_name=grade_name
        calculator.__init__(self,"Uncurved Percent Based Grade Calculator "+self.__grade_name+" Max="+str(self.__max_grade))
        
    def apply(self,a_grade_book,grade_name=None,**kwargs):
        if grade_name:
            pass
        else:
            grade_name=self.__grade_name
            
  
        #iterate through the students and grab the selected grades
        for k,a_student in a_grade_book.get_students().items():
            # get the grade object for the student
            a_grade=a_student[grade_name]


            if not a_grade.numerical():
                print (self.name()+ " Error: Did not get a numerical grade as input." + str(a_grade))
                raise Exception
    
            percent=a_grade.value()/self.__max_grade
        
            for i,v in enumerate(self.__grades_definition):
                if percent>=v[0]:
                    break
            
            #print('add Letter: '+ grade_name+" Letter" + self.__grades_definition[i][1] )

            a_student.add_grade(grade(grade_name+" Uncurved Letter",value=self.__grades_definition[i][1]),overwrite=True)

In [448]:
a_grade_book.print_students()

0 Student 0 Student Data
l1_n: 1.0
l1_1: 10.0
12_n: 7.0
l2_1: 0
l2_2: 10.0
l2_3: 10.0
l2_4: 8.0
l2_5: 10.0
l2_6: 10.0
l2_7: 10.0
l3_n: 14.0
l3_1: 9.0
l3_2: 0
l3_3: 0
l3_4: 0
l3_5: 0
l3_6: 0
l3_7: 0
l3_8: 0
l3_9: 0
l3_10: 0
l3_11: 0
l3_12: 0
l3_13: 0
l3_14: 0
l4_n: 11.0
l4_1: 0
l4_2: 0
l4_3: 0
l4_4: 0
l4_5: 0
l4_6: 0
l4_7: 0
l4_8: 0
l4_9: 0
l4_10: 0
l4_11: 0
q1_n: 1.0
q1_1: 9.5
e1_n: 15.0
e1_1: 9.0
e1_2: 9.0
e1_3: 0
e1_4: 9.0
e1_5: 8.0
e1_6: 0
e1_7: 0
e1_8: 0
e1_9: 0
e1_10: 0
e1_11: 0
e1_12: 0
e1_13: 0
e1_14: 0
e1_15: 0
l2_sum: 58.0
l1_sum: 10.0
l3_sum: 9.0
l4_sum: 0
e1_sum: 35.0
q1_sum: 9.5
l1_Uncurved Letter: A+
l2_Uncurved Letter: B-
l3_Uncurved Letter: F-
l4_Uncurved Letter: F-
q1_Uncurved Letter: A
e1_Uncurved Letter: F-
l_final: 77.0
_______________________________________
1 Student 1 Student Data
l1_n: 1.0
l1_1: 10.0
12_n: 7.0
l2_1: 0
l2_2: 0
l2_3: 0
l2_4: 0
l2_5: 0
l2_6: 0
l2_7: 0
l3_n: 14.0
l3_1: 0
l3_2: 0
l3_3: 0
l3_4: 0
l3_5: 0
l3_6: 0
l3_7: 0
l3_8: 0
l3_9: 0
l3_10: 0
l3_11: 

Test you calculator by applying the weights from the syllabus of this course and computing everyone's grade in the course.

In [ ]:
# Your solution here

In [449]:
!ls

Copy_of_Lab_7.ipynb Data1401-Grades.txt
